In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# energygenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
#                         usecols=(lambda s: s.startswith('utc') | 
#                                  s.startswith('DE_solar') |
#                                  s. startswith('DE_wind')),
#                         parse_dates=[0], index_col=0)

windgenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s. startswith('DE_wind_generation')),
                        parse_dates=[0], index_col=0)

solargenerationdata = pd.read_csv("data/time_series_60min_singleindex.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s.startswith('DE_solar')),
                        parse_dates=[0], index_col=0)

In [3]:
# weatherdata = pd.read_csv("data/weather_data.csv",
#                 usecols=(lambda s: s.startswith('utc') | 
#                          s.startswith('DE')),
#                 parse_dates=[0], index_col=0)

In [4]:
windspeed = pd.read_csv("data/weather_data.csv",
                usecols=(lambda s: s.startswith('utc') | 
                         s.startswith('DE_windspeed')),
                parse_dates=[0], index_col=0)

In [5]:
windspeed

,DE_windspeed_10m
utc_timestamp,
1980-01-01 00:00:00+00:00,5.7291
1980-01-01 01:00:00+00:00,5.7234
1980-01-01 02:00:00+00:00,5.7342
1980-01-01 03:00:00+00:00,5.7345
1980-01-01 04:00:00+00:00,5.6416
...,...
2016-12-31 19:00:00+00:00,4.2129
2016-12-31 20:00:00+00:00,4.2875
2016-12-31 21:00:00+00:00,4.2965


In [6]:
#creating training data but cutting the data based on time stamp
start_remove = pd.to_datetime('2016-01-01',utc=True)
end_remove =pd.to_datetime('2017-01-01',utc=True)

print(windspeed.index.tz)
windspeedformatted = windspeed.loc[(windspeed.index >= start_remove)]
windgenerationdataformatted = windgenerationdata.loc[(windgenerationdata.index >= start_remove) & (windgenerationdata.index < end_remove)]

UTC


In [7]:
#creating test/comparison data
start_test = pd.to_datetime('2015-01-01', utc = True)
end_test = pd.to_datetime('2016-01-01', utc = True)
windspeedtest = windspeed.loc[(windspeed.index >= start_test) & (windspeed.index < end_test)]
windgenerationtest = windgenerationdata.loc[(windgenerationdata.index >= start_test) & (windgenerationdata.index < end_test)]

In [8]:
windspeedformatted

,DE_windspeed_10m
utc_timestamp,
2016-01-01 00:00:00+00:00,3.3849
2016-01-01 01:00:00+00:00,3.3636
2016-01-01 02:00:00+00:00,3.3268
2016-01-01 03:00:00+00:00,3.2796
2016-01-01 04:00:00+00:00,3.1705
...,...
2016-12-31 19:00:00+00:00,4.2129
2016-12-31 20:00:00+00:00,4.2875
2016-12-31 21:00:00+00:00,4.2965


In [9]:
windgenerationdataformatted

,DE_wind_generation_actual
utc_timestamp,
2016-01-01 00:00:00+00:00,8579.0
2016-01-01 01:00:00+00:00,8542.0
2016-01-01 02:00:00+00:00,8443.0
2016-01-01 03:00:00+00:00,8295.0
2016-01-01 04:00:00+00:00,7319.0
...,...
2016-12-31 19:00:00+00:00,15303.0
2016-12-31 20:00:00+00:00,15354.0
2016-12-31 21:00:00+00:00,14848.0


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [11]:
#train test might not be valid for our case because we have timestamps in play
xtr, xts, ytr, yts = train_test_split(windspeedformatted,windgenerationdataformatted, test_size = 0.33)

In [15]:
#inital model training attempt
model = LinearRegression().fit(xtr,ytr)
predicted = model.predict(windspeedformatted)

In [13]:
#attempt at cross validation
from sklearn.model_selection import cross_val_score

model = LinearRegression()


scores_wind = cross_val_score(model, windspeedformatted, windgenerationdataformatted, cv=5)
print(scores_wind, "\naverage =", np.mean(scores_wind))

[0.81568392 0.58715522 0.45299934 0.61062025 0.65504413] 
average = 0.624300570072587


In [14]:
#predict using train test split data
predicted = model.predict(xts)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [16]:
predicted

array([[ 6654.2908762 ],
       [ 6579.38228282],
       [ 6449.96274121],
       ...,
       [ 9860.23799939],
       [10294.56716757],
       [10439.10910127]])

In [ ]:
#error when using model from cross validation
predicted = model.predict(windspeedformatted)

In [ ]:
#timeseries used as x axis when plotting predicted
timeseries = pd.date_range('2016-01-01', '2016-12-31 23:00:00', freq='H')

In [ ]:
timeseries

In [ ]:
windgenerationdataformatted.plot(linewidth=0.5)

In [ ]:
#I think this is an invalid test because using entire dataset in the predict but the training data is from that same set
plt.plot(timeseries,predicted, linewidth=0.5)

In [50]:
'''
Todo:
take in weather data and generation data
separate data into two sets:
Solar
X = [sun related weather columns] 
Y = wind_generation_actual
Wind:
X = [wind related weather columns] 
Y = sun_generation_actual

Trim data based on timestamp
Wind generation begins at 2010-01-01
However there are values with NaN. Chose to only take 2016 for now
No wind data past 2016

Solar generation begins at 2012-01-01

To try: if there are timestamps do I have to format data?


In [ ]:
#Solar generation and radiation

sun_start_remove = pd.to_datetime('2012-01-01',utc=True)
sun_end_remove = pd.to_datetime('2019-01-01',utc=True)
#Solar radiation
solar_radiation = pd.read_csv("data/weather_data.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s. startswith('DE_radiation')),
                        parse_dates=[0], index_col=0)
solar_radiation = solar_radiation.loc[(solar_radiation.index >= sun_start_remove) & (solar_radiation.index < sun_end_remove)]
#Solar generation
'DE_solar_generation_actual'

solar_gen_actual = solargenerationdata.loc[(solargenerationdata.index >= sun_start_remove) & (solargenerationdata.index < sun_end_remove),"DE_solar_generation_actual"]


In [ ]:
model = LinearRegression().fit(windspeedformatted,windgenerationdataformatted)

In [ ]:
#using the previously formatted data which is data from 2015
prediction = model.predict(windspeedtest)

In [ ]:
timeseries = pd.date_range('2015-01-01', '2015-12-31 23:00:00', freq='H')

In [ ]:
windgenerationtest.plot(linewidth=0.5)

In [ ]:
plt.plot(timeseries,prediction, linewidth=0.5)

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(windgenerationtest, prediction))

In [ ]:
#large number because we're deadling with large numbers
from sklearn.metrics import mean_squared_error
print(mean_squared_error(windgenerationtest, prediction))

In [ ]:
print(np.sum((windgenerationtest-prediction)**2)/(prediction.shape[0]))

In [59]:
#Solar generation and radiation

sun_start_remove = pd.to_datetime('2012-01-01',utc=True)
sun_end_remove = pd.to_datetime('2016-12-31',utc=True)
#Solar radiation
solar_radiation = pd.read_csv("data/weather_data.csv",
                        usecols=(lambda s: s.startswith('utc') | 
                                 s. startswith('DE_radiation')),
                        parse_dates=[0], index_col=0)
solar_radiation = solar_radiation.loc[(solar_radiation.index >= sun_start_remove) & (solar_radiation.index < sun_end_remove)]
#Solar generation
'DE_solar_generation_actual'

solar_gen_actual = solargenerationdata.loc[(solargenerationdata.index >= sun_start_remove) & (solargenerationdata.index < sun_end_remove),"DE_solar_generation_actual"]

In [60]:
solar_gen_actual

utc_timestamp
2012-01-01 00:00:00+00:00    NaN
2012-01-01 01:00:00+00:00    NaN
2012-01-01 02:00:00+00:00    NaN
2012-01-01 03:00:00+00:00    NaN
2012-01-01 04:00:00+00:00    NaN
                            ... 
2016-12-30 19:00:00+00:00    0.0
2016-12-30 20:00:00+00:00    0.0
2016-12-30 21:00:00+00:00    0.0
2016-12-30 22:00:00+00:00    0.0
2016-12-30 23:00:00+00:00    0.0
Name: DE_solar_generation_actual, Length: 43824, dtype: float64

In [61]:
solar_radiation

,DE_radiation_direct_horizontal,DE_radiation_diffuse_horizontal
utc_timestamp,,
2012-01-01 00:00:00+00:00,0.0,0.0
2012-01-01 01:00:00+00:00,0.0,0.0
2012-01-01 02:00:00+00:00,0.0,0.0
2012-01-01 03:00:00+00:00,0.0,0.0
2012-01-01 04:00:00+00:00,0.0,0.0
...,...,...
2016-12-30 19:00:00+00:00,0.0,0.0
2016-12-30 20:00:00+00:00,0.0,0.0
2016-12-30 21:00:00+00:00,0.0,0.0


In [64]:
sunmodel = LinearRegression()

In [65]:
xtr, xts, ytr, yts=  train_test_split(solar_radiation,solar_gen_actual, test_size = 0.33)

In [66]:
sunmodel.fit(xtr,ytr)
est_sun_generation = sunmodel.predict(xts)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').